In [1]:
import sqlalchemy as sql
import pandas as pd
import glob
from os import path, sep
import numpy as np

In [2]:
server = "thesis.ca6j6heoraog.eu-central-1.rds.amazonaws.com"
engine = sql.create_engine(f"mysql+pymysql://admin:FphvsYQek4@{server}/thesis_db")


In [3]:
con = engine.connect()

In [4]:
fnames = sorted(glob.glob('data/Time_Series_Individuals/**/Music_final_MW_HRV*.xlsx',recursive=True))
fnames[:5]

['data/Time_Series_Individuals/Music_final_MW_HRV_1001_101_BL_C_Time_Series.xlsx',
 'data/Time_Series_Individuals/Music_final_MW_HRV_1001_101_FS_C_Time_Series.xlsx',
 'data/Time_Series_Individuals/Music_final_MW_HRV_1001_101_I_C_Time_Series.xlsx',
 'data/Time_Series_Individuals/Music_final_MW_HRV_1001_102_BL_C_Time_Series.xlsx',
 'data/Time_Series_Individuals/Music_final_MW_HRV_1001_102_FS_C_Time_Series.xlsx']

In [5]:
len(fnames)

559

In [6]:
q = sql.text("""SELECT DISTINCT participant_id, group_id, series_type_id, interpolation_series_id
FROM interpolated_interval_data
WHERE interpolation_series_id=1
""")

already_in = set(tuple(r) for r in con.execute(q))

In [7]:
len(already_in)

418

In [8]:
list(already_in)[:5]

[(146, 1016, 1, 1),
 (168, 1023, 2, 1),
 (130, 1010, 3, 1),
 (242, 1047, 1, 1),
 (228, 1042, 3, 1)]

In [9]:
experiments=pd.read_sql_table('series_types',con)

experiment_ids = {
    "BL": experiments.query('experiment_name=="music" & session_type=="baseline-before"')['series_type_id'].iloc[0],
    "BLE": experiments.query('experiment_name=="music" & session_type=="baseline-after"')['series_type_id'].iloc[0],
    "I": experiments.query('experiment_name=="music" & session_type=="interaction"')['series_type_id'].iloc[0],
}
experiment_ids

{'BL': 1, 'BLE': 3, 'I': 2}

In [12]:
for f in fnames:
    skipped=0
    parts = f.split(sep)[2:]
    fname_parts= parts[-1].split('_')
    if fname_parts[0]=='EDA' or fname_parts[6]=='FS' or int(fname_parts[5])==256:
        print('skipped', fname_parts)
        continue
    key = (int(fname_parts[5]),
           int(fname_parts[4]),
           experiment_ids[fname_parts[6]],
           1) 
    if key in already_in: 
        print('skipped 2', fname_parts)
        continue
    if skipped: print("skipped", skipped)
    skipped = 0
    print(parts)
    data = pd.read_excel(f,header=None, names=['data_time','data_value'])
    data['data_value']=np.around(data.data_value,5)
    data['data_ordinal']=data.index
    data['participant_id']=fname_parts[5]
    data['group_id']=fname_parts[4]
    data['series_type_id']=experiment_ids[fname_parts[6]]
    data['interpolation_series_id'] = 1 # the code for "legacy interpolation"
    data.to_sql(name='interpolated_interval_data', con=con, if_exists='append', index=False, method='multi') 
if skipped: print("skipped", skipped)
        

skipped 2 ['Music', 'final', 'MW', 'HRV', '1001', '101', 'BL', 'C', 'Time', 'Series.xlsx']
skipped ['Music', 'final', 'MW', 'HRV', '1001', '101', 'FS', 'C', 'Time', 'Series.xlsx']
skipped 2 ['Music', 'final', 'MW', 'HRV', '1001', '101', 'I', 'C', 'Time', 'Series.xlsx']
skipped 2 ['Music', 'final', 'MW', 'HRV', '1001', '102', 'BL', 'C', 'Time', 'Series.xlsx']
skipped ['Music', 'final', 'MW', 'HRV', '1001', '102', 'FS', 'C', 'Time', 'Series.xlsx']
skipped 2 ['Music', 'final', 'MW', 'HRV', '1001', '102', 'I', 'C', 'Time', 'Series.xlsx']
skipped 2 ['Music', 'final', 'MW', 'HRV', '1001', '103', 'BL', 'C', 'Time', 'Series.xlsx']
skipped ['Music', 'final', 'MW', 'HRV', '1001', '103', 'FS', 'C', 'Time', 'Series.xlsx']
skipped 2 ['Music', 'final', 'MW', 'HRV', '1001', '103', 'I', 'C', 'Time', 'Series.xlsx']
skipped 2 ['Music', 'final', 'MW', 'HRV', '1003', '108', 'BLE', 'C', 'Time', 'Series.xlsx']
skipped 2 ['Music', 'final', 'MW', 'HRV', '1003', '108', 'BL', 'C', 'Time', 'Series.xlsx']
skipped

In [ ]:
next(iter(already_in))

In [ ]:
data

In [ ]:
fname_parts[6]

In [ ]:
fname_parts

In [ ]:
data.to_sql(name='interpolated_interval_data', con=con, if_exists='append', index=False, method='multi') 

In [13]:
sum(1 if fname_parts[0]=='EDA' or fname_parts[6]=='FS' or int(fname_parts[5])==256:
    fnames)

559